In [3]:
# pip install transformers
# pip install seqeval
# pip install pandas
# pip install numpy
# pip install matplotlib
# pip install seaborn
# !pip install scikit-learn
# !pip install torch torchvision
# !pip install nltk
# !pip install  spacy
# !pip install scipy
# !pip install yellowbrick
# !pip install tqdm
# !pip install openpyxl
# !pip install nlpaug

In [4]:
import pandas as pd
import nltk

In [5]:
path="final data.xlsx"
data=pd.read_excel(path)
data.head()

,sentence,intent
0,Please add a 'Book' node.,add node
1,Make a new node with the label 'Movie'.,add node
2,I would like to add a node labeled 'Game'.,add node
3,Can you create a node with the label 'Animal'?,add node
4,I need a new node with the label 'Plant'.,add node


In [6]:
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

### DATA PRE-PROCESSING

As BERT is trained on a corpus of full sentences of natural language and the embedding process involves contextual embedding, performing stop-word removal might interfere with the accurate functioning of BERT. So we do not remove stop words from our input. This is also why we will not be performing stemming or lemmatization.

In [7]:
import nltk
import transformers

removing all punctuation marks except '' and "" as the specifics of graph manipulation are expected to be input in quotes.

In [8]:
import string
def removePunctuation(sentence):
    cleaned="".join([char for char in sentence if char not in string.punctuation or char=="'" or char=='"'])    
    return cleaned

In [9]:
data['sentence'] =data['sentence'].apply(removePunctuation) #remove punctuation from data

convert all classes to numerical classes <br>

In [10]:
ClassNumber=1
for Class in data['intent'].unique():
    print(f'Class name:{Class}-->number: {ClassNumber}')
    data['intent']=data['intent'].replace(Class,ClassNumber)
    ClassNumber+=1

Class name:add node-->number: 1
Class name:delete node-->number: 2
Class name:add property-->number: 3
Class name:add relationship-->number: 4


C:\Users\z004z4ut\AppData\Local\Temp\ipykernel_3900\2149787421.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['intent']=data['intent'].replace(Class,ClassNumber)


**tokenization using bert tokenizer** <br>
components: <br>
-->tokenization and add special tokens thatr BERT requires and then add padding <br>
--> attention mask:1 for actual token, 0 for pad tokens<br>
-->convert tikens to ids: after you do step 1, you convert each token to unique ID<br>
-->each token is converted into a vector of (seqlen x 768) dimension <br>
--> feed the attention mask and token IDs to the model after converting them to tensors and unsqueezing them at 0th index ??? <br>
--> op= model(tokenIds,attention_mask) <br>
the op is the final embdded vector

hugging face's BERT tokenizer uses pre trained BERT base, a small version of BERT. we use cased BERT because we want our model to be case sensitive, which will be important for paramter extraction in the named entity recognition layer.

**PRE-PROCESSING FOR BERT** <br>
1. [CLS] and [SEP] are special tokens that BERT requires. this is because BERT was roginally trained for Q-A tasks. For classfn tasks, BERT requirws another special token, which is all in-built in the hugging face tokenizer. 
2. All sequences must be of the same length, so we need to do padding. We do the padding on the right side which is preferred for BERT.
3. Finally we pass in attention mask to assign importance to each token. So all non-padding tokens have an attnetion of 1 and oadding tokens have 0.
<br><br>
<i>Since BERT takes the input ids, attention mask and target classes as 3 separate outputs, we can create a dataset where each of them are a separate column to make our program more readable.<i>

In [13]:
import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
def preProcess(data):
    tokenizer=BertTokenizer.from_pretrained('bert-base-cased')
    input_ids_list=[]
    attention_masks_list=[]
    intents_list=[]
    for sentence,label in zip(data['sentence'],data['intent']):
        encoding=tokenizer.encode_plus(
            sentence,
            add_special_tokens= True,
            padding='max_length',
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='pt' #pt signifies pyTorch, use tf for tensorFlow & keras
        )
        input_ids_list.append(encoding['input_ids'])
        attention_masks_list.append(encoding['attention_mask'])
        intents_list.append(torch.tensor(label).long()) #most transformer models and BERT expect input data to be of the type LongTensor
    #to ensure that all of our tensors have the same shape we use cat()
    input_ids_list = torch.cat(input_ids_list)
    attention_masks_list = torch.cat(attention_masks_list)
    intents_list = torch.stack(intents_list)
    dataset=TensorDataset(input_ids_list,attention_masks_list,intents_list)
    return dataset

In [14]:
dataset=preProcess(data)

In [18]:
for i in range(1):
    input_ids, attention_mask, intent = dataset[i]
    print(f"Example {i}:")
    print(f"Input IDs: {input_ids}")
    print(f"Attention Mask: {attention_mask}")
    print(f"Intent: {intent}")
    print()

Example 0:
Input IDs: tensor([  101,  4203,  5194,   170,   112,  3168,   112, 14372,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,

MODEL ARCHITECTURE